<h3> StockInfo DDL</h3>

In [1]:
import pandas
from glob import glob
import numpy
import jaydebeapi

In [2]:
from pandasql import sqldf
pysqldf=lambda q: sqldf(q,globals()) #建立函式，對q做sqldf(q,globals())的意思

In [15]:
data_folder = '/Users/justin/Desktop/資料庫管理/data/IBM-Stocks'
data_files = glob('%s/IBM-20??.csv' % data_folder) #用 % 指定位址

In [16]:
df=pandas.concat((pandas.read_csv(data_file,usecols=['Date','Open','High','Low','Close','Adj Close','Volume']
, dtype={'Date':str,'Open':float,'High':float,'Low':float,'Close':float,'Adj Close':float,'Volume':int}) for data_file in data_files))

In [17]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,154.500000,154.809998,153.539993,154.250000,129.093903,4202500
1,2018-01-03,157.339996,159.809998,156.330002,158.490005,132.642441,9441600
2,2018-01-04,159.649994,162.320007,159.369995,161.699997,135.328934,7556200
3,2018-01-05,162.440002,162.899994,161.100006,162.490005,135.990097,5195800
4,2018-01-08,162.660004,163.910004,161.699997,163.470001,136.810242,5237500
...,...,...,...,...,...,...,...
246,2017-12-22,151.820007,153.000000,151.500000,152.500000,127.629303,2990600
247,2017-12-26,152.509995,153.860001,152.500000,152.830002,127.905518,2479000
248,2017-12-27,152.949997,153.179993,152.610001,153.130005,128.156570,2149300
249,2017-12-28,153.199997,154.119995,153.199997,154.039993,128.918182,2687600


In [10]:
dataframe2017=pandas.read_csv(data_files[0])
dataframe2018=pandas.read_csv(data_files[1])
dataframe2019=pandas.read_csv(data_files[2])
dataframe2020=pandas.read_csv(data_files[3])
dataframe2021=pandas.read_csv(data_files[4])
df1=dataframe2017.append(dataframe2018)
df1=df1.append(dataframe2019)
df1=df1.append(dataframe2020)
df1=df1.append(dataframe2021)
df1

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,154.500000,154.809998,153.539993,154.250000,129.093903,4202500
1,2018-01-03,157.339996,159.809998,156.330002,158.490005,132.642441,9441600
2,2018-01-04,159.649994,162.320007,159.369995,161.699997,135.328934,7556200
3,2018-01-05,162.440002,162.899994,161.100006,162.490005,135.990097,5195800
4,2018-01-08,162.660004,163.910004,161.699997,163.470001,136.810242,5237500
...,...,...,...,...,...,...,...
246,2017-12-22,151.820007,153.000000,151.500000,152.500000,127.629303,2990600
247,2017-12-26,152.509995,153.860001,152.500000,152.830002,127.905518,2479000
248,2017-12-27,152.949997,153.179993,152.610001,153.130005,128.156570,2149300
249,2017-12-28,153.199997,154.119995,153.199997,154.039993,128.918182,2687600


In [20]:
dbConnection =jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:/Users/justin/Project/h2",
    ["sa",""],
    "/Users/justin/Project/h2/bin/h2-2.1.214.jar"
)
dbCursor = dbConnection.cursor()

OSError: [Errno 0] JVM DLL not found: /opt/homebrew/Cellar/openjdk/18.0.2.1/libexec/openjdk.jdk/Contents/Home/lib/libjli.dylib


In [18]:
_sql = [
    "INSERT INTO",
    "STOCKINFO(StockID, STOCKDATE,",
    "STOCKOPEN, STOCKHIGH, STOCKLOW,",
    "STOCKCLOSE, STOCKADJCLOSE, STOCKVOL, StockNet, StockScale)",
    "VALUES('%s', '%s', %f, %f, %f, %f, %f, %f, %f, %f);"
]

sql = ' '.join(_sql)




<h3> 清除</h3>

In [19]:
dbCursor.execute('TRUNCATE TABLE STOCKINFO;')

<h3> 新增欄位</h3>

In [23]:
df1['Net']=numpy.abs(df1['Adj Close']-df1['Open'])
df1['Scale']=numpy.abs(df1['High']-df1['Low'])

<h3> 篩選資料</h3>

In [24]:
my_SQL=' SELECT SUBSTR (Date,1,7) AS YM,df2 AS data FROM df1 GROUP BY SUBSTR(Date,1,7);' 
dbCursor.execute(my_SQL)

In [ ]:
my_SQL=[
    'SELECT * FROM df1 A',
    ' INNER JOIN',
    '(SELECT SUBSTR(Date,1,7) AS YM, MAX(StocklScale) AS Scale FROM df1 GROUP BY SUBSTR(Date,1,7)) B',
    'ON A.Net = B.Net'   
]
pysqldf(' '.join(my_SQL))
dbCursor.execute(insert_sql)

In [ ]:
my_SQL=[
    'SELECT * FROM df1 A',
    ' INNER JOIN',
    '(SELECT SUBSTR(Date,1,7) AS YM, MIN(StockNet) AS Net FROM df1 GROUP BY SUBSTR(Date,1,7)) B',
    'ON A.Net = B.Net'   
]
pysqldf(' '.join(my_SQL))
dbCursor.execute(insert_sql)

In [ ]:
my_monly_stats=(my_monthly['Volume'].mean())
my_SQL_statement=my_SQL % str(my_monly_stats[0])
my_SQL_statement